In [1]:
%matplotlib inline
from pylab import *

import quandl
import plotly
from datetime import datetime

import pandas as pd

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go


def get_date(date, offset=0):
    if offset == 1:
        return pd.to_datetime(datetime.strptime(date, '%Y-%m-%d')) + pd.DateOffset(days=1)
    elif offset == -1:
        return pd.to_datetime(datetime.strptime(date, '%Y-%m-%d')) + pd.DateOffset(days=-1)
    if offset == 0:
        return pd.to_datetime(datetime.strptime(date, '%Y-%m-%d'))


# Quandl API's key:
# fKcJbMJjimNnxHBto1Mo
quandl_key = 'fKcJbMJjimNnxHBto1Mo'
quandl.ApiConfig.api_key = quandl_key

# Plotly API's key
plotly_key = 'dvx0rb722c'
plotly.tools.set_credentials_file(username='devpog', api_key=plotly_key)

#ind_dict = {'dow': 'BCB/UDJIAD1', 'sp500': 'YAHOO/INDEX_GSPC', 'nasdaq':'NASDAQOMX/COMP'}
ind_dict = {'sp500': 'YAHOO/INDEX_GSPC', 'nasdaq':'NASDAQOMX/COMP'}

election_dates = {2016: '2016-11-08',
                  2012: '2012-11-06',
                  2008: '2008-11-04',
                  2004: '2004-11-02',
                  2000: '2000-11-07',
                  1996: '1996-11-05',
                  1992: '1992-11-05'}

candidates = {2016: 'D Trump VS H Clinton',
              2012: 'B Obama VS M Romney',
              2008: 'B Obama VS J McCain',
              2004: 'GW Bush VS J Kerry',
              2000: 'GW Bush VS A Gore',
              1996: 'W Clinton VS B Dole',
              1992: 'W Clinton VS G Bush'}

election_dates_tr = dict((k, get_date(v)) for (k, v) in election_dates.items())
election_next_day = dict((k, get_date(v, 1)) for (k, v) in election_dates.items())
election_prior_day = dict((k, get_date(v, -1)) for (k, v) in election_dates.items())

colors = {'dow': 'blue', 'nasdaq': 'green', 'sp500': 'red'}

update_data = True
if update_data:
    indices = dict()
    for k, v in ind_dict.items():
        df = pd.DataFrame(quandl.get(v))
        if k == 'dow':
            df = df.loc[:, ['Value']]
            df = df.rename(columns={'Value': 'Open'})
        elif k == 'nasdaq':
            df = df.loc[:, ['Index Value']]
            df = df.rename(columns={'Index Value': 'Open'})
        elif k == 'sp500':
            df = df.loc[:, ['Open']]
        indices[k] = df

traces = []
for k, v in indices.items():
    reduced = indices[k]['19920101':]
    ind = pd.to_datetime(pd.Series(reduced.index).map(lambda d: datetime.strftime(d, '%Y-%m-%d')))
    ts = pd.Series([i for l in reduced.loc[:, ['Open']].values for i in l])
    traces.append(go.Scatter(x=ind, y=ts, name=k, mode='line'))

shapes = []
for k, v in election_dates_tr.items():
    for e, i in enumerate(ind):
        if i == v:
            shapes.append(dict(type='line',
                               fillcolor='red',
                               x0=i, x1=i,
                               y0=0, y1=max(ts)))


annotations = []
for k, v in election_dates_tr.items():
    for i in ind:
        if i == v:
            annotations.append(dict(x=i,
                                    y=300,
                                    xref='x',
                                    yref='y',
                                    text=candidates[k]))

layout = go.Layout(
    showlegend=True,
    annotations=annotations,
    shapes=shapes
)

fig = go.Figure(data=traces, layout=layout)
url = py.iplot(fig, filename='pandas-time-series')